In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph
from sparse_util import dilate, zhangSuen
from realign import realign
from util import get_path
import pandas as pd

In [3]:
date0='0709_1157'
date1='0709_1557'
date2='0709_1934'
plate=13

In [4]:
dates=[date0,date1,date2]
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [5]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [6]:
skeleton_docs=[sparse.dok_matrix(dic_skel) for dic_skel in dic_skels]

In [7]:
skeleton_docs_aligned = [skeleton_docs[0],realign(skeleton_docs[1],skeleton_docs[0]),realign(skeleton_docs[2],skeleton_docs[0])]

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1988847 1981966
253504 875
245 14
3 4
1 1
0 0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1995905 1988986
252356 1016
362 63
36 32
0 0


In [8]:
graph_tabs=[from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_aligned]

In [9]:
nx_graph_poss=[generate_nx_graph(graph_tab) for graph_tab in graph_tabs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [11]:
sparse.save_npz(f'Data/skeleton_{date1}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[1]))

In [12]:
sparse.save_npz(f'Data/skeleton_{date2}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[2]))

In [13]:
sparse.save_npz(f'Data/skeleton_{date0}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[0]))

In [ ]:
skeleton_doc_1_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date1}_{plate}_full.npz'))
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)
anx_graph_t0m1=prune_graph(nx_graph_1)

In [14]:
anx_graph_t0m2,anx_graph_t0m1,anx_graph_t0=nx_graph_pruned[0], nx_graph_pruned[1], nx_graph_pruned[2]
apos_t0m2,apos_t0m1,apos_t0 = poss_aligned[0], poss_aligned[1], poss_aligned[2]
skeleton_doc_0_thinned,skeleton_doc_1_thinned, skeleton_doc_2_thinned= skeleton_docs_aligned[0], skeleton_docs_aligned[1], skeleton_docs_aligned[2]

In [ ]:
apos_t0,anx_graph_t0,anx_graph_t0m1,corresp_tips=second_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)
apos_t0m1,anx_graph_t0m1,anx_graph_t0m2,corresp_tips=second_identification(anx_graph_t0m2,anx_graph_t0m1,apos_t0m2,apos_t0m1,50)

In [ ]:
begin=15000
end=20000 

In [ ]:
skeleton_t0m2=generate_skeleton(anx_graph_t0m2,skeleton_doc_0_thinned.shape)

In [ ]:
skeleton_t0m1=generate_skeleton(anx_graph_t0m1,skeleton_doc_1_thinned.shape)

In [ ]:
skeleton_t0=generate_skeleton(anx_graph_t0,skeleton_doc_2_thinned.shape)

In [ ]:
tab_labeled_t0m2 = from_nx_to_tab(anx_graph_t0m2,apos_t0m2)
tab_labeled_t0m1 = from_nx_to_tab(anx_graph_t0m1,apos_t0m1)
tab_labeled_t0 = from_nx_to_tab(anx_graph_t0,apos_t0)

In [ ]:
graph_t0m2=from_sparse_to_graph(skeleton_t0m2)
graph_t0m1=from_sparse_to_graph(skeleton_t0m1)
graph_t0=from_sparse_to_graph(skeleton_t0)

In [ ]:
tab_labeled_t0m2.to_csv(get_path(date0,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0m1.to_csv(get_path(date1,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0.to_csv(get_path(date2,plate,True,extension='_full_labeled.csv'))

In [ ]:
graph_t0m2.to_csv(get_path(date0,plate,True,extension='_full.csv'))
graph_t0m1.to_csv(get_path(date1,plate,True,extension='_full.csv'))
graph_t0.to_csv(get_path(date2,plate,True,extension='_full.csv'))

In [ ]:
tab_labeled_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full_labeled.csv')
tab_labeled_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full_labeled.csv')
tab_labeled_t0.to_csv(f'Data/graph_{date2}_{plate}_full_labeled.csv')

In [ ]:
graph_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full.csv')
graph_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full.csv')
graph_t0.to_csv(f'Data/graph_{date2}_{plate}_full.csv')

In [3]:
tab_labeled_t0m2=pd.read_csv(f'Data/graph_{date0}_{plate}_full_labeled.csv')
tab_labeled_t0m1=pd.read_csv(f'Data/graph_{date1}_{plate}_full_labeled.csv')
tab_labeled_t0=pd.read_csv(f'Data/graph_{date2}_{plate}_full_labeled.csv')

In [5]:
graph_t0m2=pd.read_csv(f'Data/graph_{date0}_{plate}_full.csv')
graph_t0m1=pd.read_csv(f'Data/graph_{date1}_{plate}_full.csv')
graph_t0=pd.read_csv(f'Data/graph_{date2}_{plate}_full.csv')

In [ ]:
tab_labeled_t0m2.loc[tab_labeled_t0m2['origin_label']==300]

In [ ]:
tab_labeled_t0m1.loc[tab_labeled_t0m1['origin_label']==300]

In [180]:
sub_skeleton_t0m1=skeleton_t0m1[begin:end,begin:end]


In [181]:
sub_skeleton_t0=skeleton_t0[begin:end,begin:end]

In [182]:
list_node_t0m1=[node for node in anx_graph_t0m1 if (begin<pos_1[node][0]<end and begin<pos_1[node][1]<end)]

In [183]:
list_node_t0=[node for node in anx_graph_t0 if (begin<apos_t0[node][0]<end and begin<apos_t0[node][1]<end)]

In [185]:
plot_t_tp1(list_node_t0m1,list_node_t0,pos_1,apos_t0,sub_skeleton_t0m1.todense(),sub_skeleton_t0.todense(),shift=(begin,begin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
plot_t_tp1([],[],pos_1,apos_t0,skeleton1.todense(),skeleton2.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
np.ceil(4.3)

5.0

In [41]:
from scipy.optimize import minimize

In [107]:
sub_skeleton_t0m1.shape

(5000, 5000)

In [172]:
shift_sub_skeleton_t0m1=shift_skeleton(sub_skeleton_t0m1,shift)

In [104]:
np.round(5.4)

5.0

In [ ]:
def shift_skeleton(skeleton,shift):
    shifted_skeleton=sparse.dok_matrix(skeleton.shape, dtype=bool)
    for pixel in skeleton.keys():
#             print(pixel[0]+shift[0],pixel[1]+shift[1])
            if (skeleton.shape[0]>np.ceil(pixel[0]+shift[0])>0 and skeleton.shape[1]>np.ceil(pixel[1]+shift[1])>0):
                shifted_pixel = (np.round(pixel[0]+shift[0]),np.round(pixel[1]+shift[1]))
                shifted_skeleton[shifted_pixel]=1
    return(shifted_skeleton)

In [40]:
skeleton_t0-skeleton_t0m1

<20687x45556 sparse matrix of type '<class 'numpy.bool_'>'
	with 2683488 stored elements in Compressed Sparse Row format>